In [2]:
from google.cloud import bigquery
from google.oauth2 import service_account
import json
import requests

### On appelle les données de config

headers = {'Content-Type': 'application/json'}

CRED_PATH = "flash-datum-368412-1df83d779e92.json"
LOCATION = "us-central1"
CREDENTIALS = service_account.Credentials.from_service_account_file(CRED_PATH)


def get_project_id():
    with open(CRED_PATH) as f:
        data = json.load(f)
        return data['project_id']

In [3]:
PROJECT_ID = get_project_id()
CLIENT = bigquery.Client(project=PROJECT_ID, credentials=CREDENTIALS)

In [4]:
pip install db-dtypes

Note: you may need to restart the kernel to use updated packages.


In [5]:
query = f"""
SELECT
start_station_name
, AVG(duration) AS duration
FROM `bigquery-public-data`.london_bicycles.cycle_hire
GROUP BY start_station_name
"""
data = CLIENT.query(query = query).to_dataframe()

In [6]:
data

,start_station_name,duration
0,"Cheapside, Bank",1010.454593
1,"Bath Street, St. Luke's",822.541242
2,"Queen Victoria Street, St. Paul's",1247.469940
3,"Finsbury Circus, Liverpool Street",921.608733
4,"Westfield Library Corner, Shepherd's Bush",2486.070266
...,...,...
875,"Monier Road, Newham",1087.058824
876,"Here East South, Queen Elizabeth Olympic Park",1673.258427
877,"Contact Centre, Southbury House",4364.000000
878,Monier Road,1576.363636


In [7]:
query = f"""
SELECT
EXTRACT(dayofweek FROM start_date) AS dayofweek
, AVG(duration) AS duration
FROM `bigquery-public-data`.london_bicycles.cycle_hire
GROUP BY dayofweek
"""
data = CLIENT.query(query = query).to_dataframe()

In [8]:
data

,dayofweek,duration
0,7,1754.172989
1,2,1188.622889
2,6,1253.651453
3,4,1154.297052
4,5,1190.617767
5,3,1111.307994
6,1,1866.445302


In [9]:
query = f"""
SELECT
bikes_count
, AVG(duration) AS duration
FROM `bigquery-public-data`.london_bicycles.cycle_hire
JOIN `bigquery-public-data`.london_bicycles.cycle_stations
ON cycle_hire.start_station_name = cycle_stations.name
GROUP BY bikes_count
"""
data = CLIENT.query(query = query).to_dataframe()

In [10]:
data

,bikes_count,duration
0,6,1302.229721
1,7,1201.877221
2,11,1151.064789
3,10,1241.416437
4,0,1111.002412
5,19,1214.423128
6,9,1223.534636
7,2,1403.700132
8,25,1336.687940
9,18,1269.005728


In [11]:
query = f"""
SELECT
CORR(bikes_count, duration) AS corr
FROM `bigquery-public-data`.london_bicycles.cycle_hire
JOIN `bigquery-public-data`.london_bicycles.cycle_stations
ON cycle_hire.start_station_name = cycle_stations.name
"""
data = CLIENT.query(query = query).to_dataframe()

In [12]:
data

,corr
0,0.002122


In [13]:
query = f"""
SELECT
duration
, start_station_name
, CAST(EXTRACT(dayofweek FROM start_date) AS STRING) as dayofweek
, CAST(EXTRACT(hour FROM start_date) AS STRING) AS hourofday
FROM `bigquery-public-data`.london_bicycles.cycle_hire
"""
data = CLIENT.query(query = query).to_dataframe()

In [14]:
data

,duration,start_station_name,dayofweek,hourofday
0,1260,"Moor Street, Soho",1,2
1,2280,"Moor Street, Soho",7,2
2,540,"Eversholt Street , Camden Town",7,3
3,1500,"World's End Place, West Chelsea",1,3
4,1080,"Devonshire Terrace, Bayswater",6,4
...,...,...,...,...
24369196,2400,"Jubilee Plaza, Canary Wharf",7,23
24369197,600,"Belvedere Road, South Bank",5,23
24369198,1080,"Bury Place, Holborn",5,23
24369199,1020,"Red Lion Street, Holborn",7,23


In [36]:
query = f"""
CREATE OR REPLACE MODEL bqml.bicycle_model
OPTIONS(input_label_cols=['duration'], model_type='linear_reg')
AS
SELECT
duration
, start_station_name
, CAST(EXTRACT(dayofweek FROM start_date) AS STRING) as dayofweek
, CAST(EXTRACT(hour FROM start_date) AS STRING) AS hourofday
FROM `bigquery-public-data`.london_bicycles.cycle_hire
"""
CLIENT.query(query = query)

QueryJob<project=flash-datum-368412, location=EU, id=998e2a17-9fb8-4bb5-acaa-382a694b915f>

In [40]:
#Evaluating the model
query = f"""
SELECT * FROM ML.EVALUATE(MODEL bqml.bicycle_model)
"""
data = CLIENT.query(query = query).to_dataframe()

In [41]:
data

,mean_absolute_error,mean_squared_error,mean_squared_log_error,median_absolute_error,r2_score,explained_variance
0,1025.364041,1.862628e+08,0.862452,542.099872,0.003625,0.003646


In [42]:
#Combining days of the week
query = f"""
CREATE OR REPLACE MODEL bqml.bicycle_model_weekday
OPTIONS(input_label_cols=['duration'], model_type='linear_reg')
AS
SELECT
duration
, start_station_name
, IF(EXTRACT(dayofweek FROM start_date) BETWEEN 2 and 6,
'weekday', 'weekend') as dayofweek
, CAST(EXTRACT(hour FROM start_date) AS STRING) AS hourofday
FROM `bigquery-public-data`.london_bicycles.cycle_hire
"""
CLIENT.query(query = query)

QueryJob<project=flash-datum-368412, location=EU, id=3b0d6b24-efa4-4931-9aac-36c375466003>

In [43]:
#Combining days of the week
query = f"""
CREATE OR REPLACE MODEL bqml.bicycle_model_bucketized
OPTIONS(input_label_cols=['duration'], model_type='linear_reg')
AS
SELECT
duration
, start_station_name
, IF(EXTRACT(dayofweek FROM start_date) BETWEEN 2 and 6, 'weekday',
'weekend')
as dayofweek
, ML.BUCKETIZE(EXTRACT(hour FROM start_date), [5, 10, 17]) AS hourofday
FROM `bigquery-public-data`.london_bicycles.cycle_hire
"""
CLIENT.query(query = query)

QueryJob<project=flash-datum-368412, location=EU, id=f77ccbc9-929a-463b-aef6-f17f0be28379>

In [45]:
#Predicting with the Model
query = f"""
SELECT * FROM ML.PREDICT(MODEL bqml.bicycle_model_bucketized,
(SELECT 'Park Lane , Hyde Park' AS start_station_name
, 'weekday' AS dayofweek, '17' AS hourofday)
)
"""
data = CLIENT.query(query = query).to_dataframe()

In [46]:
data

,predicted_duration,start_station_name,dayofweek,hourofday
0,-1411.017197,"Park Lane , Hyde Park",weekday,17


In [49]:
query = f"""
CREATE OR REPLACE MODEL bqml.bicycle_model_bucketized
TRANSFORM(* EXCEPT(start_date)
, IF(EXTRACT(dayofweek FROM start_date) BETWEEN 2 and 6,
'weekday', 'weekend') as dayofweek
, ML.BUCKETIZE(EXTRACT(HOUR FROM start_date), [5, 10, 17]) AS
hourofday
)
OPTIONS(input_label_cols=['duration'], model_type='linear_reg')
AS
SELECT
duration
, start_station_name
, start_date
FROM `bigquery-public-data`.london_bicycles.cycle_hire
"""
CLIENT.query(query = query)

QueryJob<project=flash-datum-368412, location=EU, id=ee861639-6d89-4820-83b2-ffff40ffd29d>

In [50]:
#Predicting with the Model
query = f"""
SELECT * FROM ML.PREDICT(MODEL bqml.bicycle_model_bucketized,
(SELECT 'Park Lane , Hyde Park' AS start_station_name
, CURRENT_TIMESTAMP() AS start_date)
)
"""
data = CLIENT.query(query = query).to_dataframe()

In [51]:
data

,predicted_duration,start_station_name,start_date
0,2765.261775,"Park Lane , Hyde Park",2023-01-14 21:54:39.318751+00:00


In [53]:
query = f"""
DECLARE tomorrow_3am TIMESTAMP;
SET tomorrow_3am = TIMESTAMP_ADD(
TIMESTAMP(DATE_ADD(CURRENT_DATE(), INTERVAL 1 DAY)),
INTERVAL 3 HOUR);
WITH generated AS (
SELECT
name AS start_station_name
, GENERATE_TIMESTAMP_ARRAY(
tomorrow_3am,
TIMESTAMP_ADD(tomorrow_3am, INTERVAL 24 HOUR),
INTERVAL 1 HOUR) AS dates
FROM
`bigquery-public-data`.london_bicycles.cycle_stations
),
features AS (
SELECT
start_station_name
, start_date
FROM
generated
, UNNEST(dates) AS start_date
)
SELECT * FROM ML.PREDICT(MODEL bqml.bicycle_model_bucketized,
(SELECT * FROM features)
)
"""
data=CLIENT.query(query = query).to_dataframe()

In [54]:
data

,predicted_duration,start_station_name,start_date
0,2348.370557,"Queen's Gate (North), Kensington",2023-01-15 03:00:00+00:00
1,2348.370557,"Queen's Gate (North), Kensington",2023-01-15 04:00:00+00:00
2,1385.250742,"Queen's Gate (North), Kensington",2023-01-15 05:00:00+00:00
3,1385.250742,"Queen's Gate (North), Kensington",2023-01-15 06:00:00+00:00
4,1385.250742,"Queen's Gate (North), Kensington",2023-01-15 07:00:00+00:00
...,...,...,...
19895,2020.097723,"Edgware Road Station, Marylebone",2023-01-15 23:00:00+00:00
19896,2378.685798,"Edgware Road Station, Marylebone",2023-01-16 00:00:00+00:00
19897,2378.685798,"Edgware Road Station, Marylebone",2023-01-16 01:00:00+00:00
19898,2378.685798,"Edgware Road Station, Marylebone",2023-01-16 02:00:00+00:00


In [55]:
#Model Weights
query = f"""
SELECT * FROM ML.WEIGHTS(MODEL bqml.bicycle_model_bucketized)
"""
data = CLIENT.query(query = query).to_dataframe()

In [56]:
data

,processed_input,weight,category_weights
0,start_station_name,NaN,"[{'category': 'Wellington Arch, Hyde Park', 'w..."
1,dayofweek,NaN,"[{'category': 'weekday', 'weight': -374.950354..."
2,hourofday,NaN,"[{'category': 'bin_2', 'weight': -2841.0867429..."
3,__INTERCEPT__,4034.739846,[]
